In [1]:
#This is a notebook for the 3rd week's assignment of capstone project.
#It is about wrangling,segmenting, and clustering initially locations 
#in Canada, then neighborhoods of Toronto. 

In [2]:
#Required libraries FOR:

#data processing
import numpy as np
import pandas as pd

#web scraping
import urllib.request
from bs4 import BeautifulSoup

#plotting maps
#!conda install -c conda-forge folium --yes
#I have Ubuntu on my laptop so 
!pip3 install folium
import folium

#retrieving latitude-longitude of Neighbors
from geopy.geocoders import Nominatim

#clustering
!pip3 install --upgrade scipy
!pip3 install --upgrade scikit-learn
from sklearn.cluster import KMeans

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scipy in /home/peker/.local/lib/python3.6/site-packages (1.4.1)
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn in /home/peker/.local/lib/python3.6/site-packages (0.23.1)


In [3]:
#Create a soup object from given url 
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

In [4]:
#Find all tables in the url, then pick the desired one
all_tables=soup.find_all("table")
#all_tables

right_table=soup.find('table', class_='wikitable sortable')
#right_table

In [5]:
#Initiate arrays for storing contents of table
po_codes=[]
boroghs=[]
n_hoods=[]

In [6]:
#Populate arrays with data extracted from wiki table.
#To trim new line marks, use slicing [:-1] 
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        po_codes.append(cells[0].find(text=True)[:-1])
        boroghs.append(cells[1].find(text=True)[:-1])
        n_hoods.append(cells[2].find(text=True)[:-1])

In [7]:
#Initiate an empty dataframe with desired column names
column_names = ['PostalCode', 'Borough', 'Neighborhood']
ns_df = pd.DataFrame(columns=column_names)

In [8]:
#Populate the dataframe with the extracted data
ns_df['PostalCode']=po_codes
ns_df['Borough']=boroghs
ns_df['Neighborhood']=n_hoods

In [9]:
#Remove rows without a Borough name
ns_df = ns_df.drop(ns_df[ns_df['Borough'] == 'Not assigned'].index).reset_index(drop=True)
ns_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [10]:
#Check if dataframe has a borough but a Not assigned neighborhood.
#This is done by first locating NULL elements in dataframe 'Neighborhood'
#column. Following .isnull test, we create a list from indices of elements
#in test with 'False' value and call it check list. Finally we compare 
#lengths of test list and check list. In case comparison is 'True', we have
#no NULL elements. If comparison gives 'False', we have Not assigned 
#neighborhoods.
test = pd.isnull(ns_df.loc[:, 'Neighborhood'])
check = np.where(test==False)[0]
len(test)==len(check)

True

In [11]:
#Check the shape of dataframe.
ns_df.shape

(103, 3)

In [12]:
#Get latitude longitude data from csv file.
all_locs=pd.read_csv('Geospatial_Coordinates.csv')

In [13]:
#Create two empty lists for storing latitude/longitude data
latitude=[]
longitude=[]

In [14]:
#Populate lists with the data from csv file.
for i in range(len(ns_df)): 
    mark = ns_df['PostalCode'][i]
    mark_val=all_locs[all_locs['Postal Code']==mark]
    latitude += [mark_val.iloc[0]['Latitude'] ]
    longitude += [mark_val.iloc[0]['Longitude']]

In [15]:
#Add 'Latitude' and 'Longitude' columns/values to dataframe
ns_df['Latitude']=latitude
ns_df['Longitude']=longitude
ns_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [16]:
#Here we plot all neighborhoods of Toronto
map_toronto_allNeighs = folium.Map(location=[latitude[0], longitude[0]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(ns_df['Latitude'], ns_df['Longitude'], ns_df['Borough'], ns_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_allNeighs)  
    
map_toronto_allNeighs

In [17]:
#Here we pick only 'Toronto' including boroghs. There is a regex way of doing it
#but I am lazy.
downtown_toronto = ns_df[ns_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
central_toronto = ns_df[ns_df['Borough'] == 'Central Toronto'].reset_index(drop=True)
east_toronto = ns_df[ns_df['Borough'] == 'East Toronto'].reset_index(drop=True)
west_toronto = ns_df[ns_df['Borough'] == 'West Toronto'].reset_index(drop=True)
ton_df=pd.concat([downtown_toronto,central_toronto,east_toronto,west_toronto])
ton_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [18]:
#ton_df has multiple Neighborhoods per Borogh. This will complicate 
#later clustering work. Therefore, we will explode Neighborhood column
#to individual elements while keeping associated Borogh and Latitude/
#Longitude information. 
ton_df = \
(ton_df.set_index(ton_df.columns.drop('Neighborhood',0).tolist())
   .Neighborhood.str.split(', ', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'Neighborhood'})
   .loc[:, ns_df.columns]
)
ton_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
3,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
4,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
5,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
6,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
9,M6G,Downtown Toronto,Christie,43.669542,-79.422564


In [19]:
map_toronto_Toronto = folium.Map(location=[latitude[0], longitude[0]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(ton_df['Latitude'], ton_df['Longitude'], ton_df['Borough'], ton_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_Toronto)  
    
map_toronto_Toronto

In [20]:
#Foursquare base info
CLIENT_ID = 'RYCX141XUBTJKRVJMLA51KZVARP4KANFZDUVGY4SNPBT0JV0' # your Foursquare ID
CLIENT_SECRET = 'BRS3PNN0IUEIUAJ4LYDWJ0EWFVVA22FKZMCUHC4331QBADIH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RYCX141XUBTJKRVJMLA51KZVARP4KANFZDUVGY4SNPBT0JV0
CLIENT_SECRET:BRS3PNN0IUEIUAJ4LYDWJ0EWFVVA22FKZMCUHC4331QBADIH


In [21]:
ton_df.loc[:10, 'Neighborhood']

0                       Regent Park
1                      Harbourfront
2                      Queen's Park
3     Ontario Provincial Government
4                   Garden District
5                           Ryerson
6                    St. James Town
7                       Berczy Park
8                Central Bay Street
9                          Christie
10                         Richmond
Name: Neighborhood, dtype: object

In [22]:
ton_df.shape

(75, 5)

In [23]:
#Geocode is not able to find locations of some Neighborhoods, so I will drop them from
#the list before analysis.
black_list=['Ontario Provincial Government','Railway Lands','Island airport','Stn A PO Boxes', 
           'Business reply mail Processing Centre']
for i in black_list:
    ton_df = ton_df.drop(ton_df[ton_df['Neighborhood'] == i].index).reset_index(drop=True)

In [26]:
#Cleaned up list is ready for the use in geocode. Now we extract location data for
#Neighboors.
city = 'Toronto, Canada'
boroughs  = ton_df['Neighborhood']
geolocator = Nominatim(user_agent="toron_explorer")
i=0
for borough in boroughs:
    address = borough + ', ' + city
    location = geolocator.geocode(address)
    lat = location.latitude
    lng = location.longitude
    print(i,address, lat, lng)
    i+=1

0 Regent Park, Toronto, Canada 43.6607056 -79.3604569
1 Harbourfront, Toronto, Canada 43.6400801 -79.3801495
2 Queen's Park, Toronto, Canada 43.659659 -79.3903399
3 Garden District, Toronto, Canada 43.6564995 -79.3771141
4 Ryerson, Toronto, Canada 43.65846945 -79.37899327245886
5 St. James Town, Toronto, Canada 43.6694032 -79.3727041
6 Berczy Park, Toronto, Canada 43.64798435 -79.37539591138858
7 Central Bay Street, Toronto, Canada 43.6607076 -79.3858018
8 Christie, Toronto, Canada 43.6641106 -79.4184051
9 Richmond, Toronto, Canada 43.8125891 -79.2633703893962
10 Adelaide, Toronto, Canada 43.6504863 -79.37949791351815
11 King, Toronto, Canada 43.6489494 -79.3777538
12 Harbourfront East, Toronto, Canada 43.6400801 -79.3801495
13 Union Station, Toronto, Canada 43.6446934 -79.38013200585965
14 Toronto Islands, Toronto, Canada 43.6230544 -79.39431563721035
15 Toronto Dominion Centre, Toronto, Canada 43.6473768 -79.3813719429129
16 Design Exchange, Toronto, Canada 43.6477412 -79.3801145
17 